In [247]:
import mlflow
import mlflow.pytorch
import tempfile

In [ ]:
mlrun = './mlruns/0/'

In [ ]:
! ls ./mlruns/0

In [ ]:
import yaml
import os
import glob

In [ ]:
with open(os.path.join(mlrun, 'meta.yaml'), 'r') as f:
    meta = yaml.safe_load(f)

In [ ]:
meta

In [ ]:
glob.glob(os.path.join(mlrun, '*'))

In [ ]:
rundir = './mlruns/0/f63a82ab86524a7c8708310d61c54431/'

In [ ]:
glob.glob(rundir)

In [ ]:
with open(os.path.join(rundir, 'meta.yaml'), 'r') as f:
    runmeta = yaml.safe_load(f)

In [ ]:
runmeta

In [ ]:
! ls ./mlruns/0/f63a82ab86524a7c8708310d61c54431/artifacts/path/data/

In [ ]:
def read_params(run_path: str):
    params = dict()
    for paramName in glob.glob(os.path.join(run_path, 'params/*')):
        print(paramName)
        with open(paramName, 'r') as f:
            value = f.read()
        params[os.path.basename(paramName)]=value
    return params

In [ ]:
read_params(run_path='./mlruns/0/f63a82ab86524a7c8708310d61c54431')

## using client

In [ ]:
# get_run(run_id: str) 
from mlflow.client import MlflowClient

In [142]:
import mlflow.pytorch

import mlflow.onnx

In [ ]:
local_client = MlflowClient()

In [85]:
experiment = local_client.get_experiment(experiment_id='0')

In [124]:
runs = local_client.search_runs(experiment_ids=[experiment.experiment_id]) #, order_by='start_time DESC'

In [125]:
for run in runs:
    print(run.info.run_id)

e10f5cc7f8c44f079fd8214d4f14b840
7d8add9e6d6b424e9aef34c3a8b84c56
fba2fecd08e4444098a0a9b81c8759d7
21fe09e3049a4ef491b32124041ba10d
f63a82ab86524a7c8708310d61c54431
943e0331574c496a8d4133039c547f57
faad19c43a20438ba1d392957ac870be
6f85e591e09a42dcbc225539359fca23
a4217081db4d41c9b4c2deb2844f44a1
481dcf3917214c4e8b2d5ff94244d168
5eca6cb9aa0f452a8991f444a8dccc8f
ae450efcf5114dc1b2fecbc129107c4b
ea63cb7ec24d4ca0a63c3094dfa5c5e7
2b2ab36e96c0465f8e00362e0d3f8d2e
1e6c9f06033343a79b6c26c397fb3cba
7f2fd88ec2f9435faca1bcc0e803ac92
489269e357cc4ecb8a9411b2b9310f31
e6a42d424a7640daac859de008c62490
3d787a300d6a4cf7bd5bfee483baf2ba


In [126]:
r = runs[0]

In [115]:
# client.log_batch

In [129]:
r.info.run_id

'e10f5cc7f8c44f079fd8214d4f14b840'

In [131]:
r.data.params

{}

In [132]:
r.info.artifact_uri

'file:///home/jj/git/modelrepository/notebooks/mlruns/0/e10f5cc7f8c44f079fd8214d4f14b840/artifacts'

In [135]:
! ls /home/jj/git/modelrepository/notebooks/mlruns/0/e10f5cc7f8c44f079fd8214d4f14b840/artifacts/

conda.yaml  data  MLmodel  python_env.yaml  requirements.txt


In [89]:
# this does not work
#local_client.get_registered_model(name='suave-cat-581')
#local_client.search_registered_models()

In [158]:
r.par

<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "e10f5cc7f8c44f079fd8214d4f14b840", '
                             '"artifact_path": "", "utc_time_created": '
                             '"2022-11-02 10:03:15.314602", "flavors": '
                             '{"pytorch": {"model_data": "data", '
                             '"pytorch_version": "1.11.0+cu102", "code": '
                             'null}, "python_function": {"pickle_module_name": '
                             '"mlflow.pytorch.pickle_module", "loader_module": '
                             '"mlflow.pytorch", "python_version": "3.10.6", '
                             '"data": "data", "env": "conda.yaml"}}, '
                             '"model_uuid": '
                             '"f4cc3196224b482ca49f503c49485c98", '
                             '"mlflow_version": "1.30.0"}]',
 'mlflow.runName': 'burly-turtle-743',
 'mlflow.source.name': '/home/jj/anaconda3/envs/wheat/lib/

In [139]:
a_model = mlflow.pytorch.load_model(r.info.artifact_uri)

In [140]:
a_model

SimpleNet(
  (l1): Linear(in_features=784, out_features=10, bias=True)
)

In [257]:
remote_client.create_run?

In [249]:
local_client.search_experiments()

[<Experiment: artifact_location='file:///home/jj/git/modelrepository/notebooks/mlruns/0', creation_time=None, experiment_id='0', last_update_time=None, lifecycle_stage='active', name='Default', tags={}>]

In [165]:
experiments_ids = [exp.experiment_id for exp in experiments]

In [ ]:

for experiment in local_client.search_experiments():
    print('Processing experiment ', experiment.experiment_id, experiment.name)
    
    #check if it already exists:
    lst = remote_client.search_experiments(filter_string=f"name = '{experiment.name}'")
    if not lst:
        print("Creating experiment")
        remote_id = remote_client.create_experiment(experiment.name)#, artifact_location='http://localhost:5000/')
         
    else:
        print("Experiment exists")
        remote_id = lst[0].experiment_id
        
    print("Remote experiment id", remote_id)

    runs = local_client.search_runs(experiment_ids=[experiment.experiment_id])

    for run in runs:
        remote_run = remote_client.create_run(experiment_id=remote_id, start_time=run.info.start_time)

        remote_run_id = remote_run.info.run_id
        print('Processing run:', run.info.run_id, '->', remote_run_id)

        for metric in run.data.metrics:
            metric_history = local_client.get_metric_history(run_id=run.info.run_id, key=metric)
            print(f"Got metric history for {metric} length={len(metric_history)}")
            remote_client.log_batch(run_id=remote_run_id, metrics=metric_history)

        
        print('Params:', run.data.params)
        for n,v in run.data.params.items():
            remote_client.log_param(run_id=remote_run_id, key=n, value=v)
        
        #tags if any?

        artifacts = local_client.list_artifacts(run_id=run.info.run_id)
        #mlflow.set_tracking_uri('http://localhost:5000')
        with tempfile.TemporaryDirectory() as tmpdirname:
            print('created temporary directory', tmpdirname)
            for art in artifacts:
                local_client.download_artifacts(run_id=run.info.run_id, path=art.path, dst_path=tmpdirname)
                remote_client.log_artifact(run_id=remote_run_id, local_path=os.path.join(tmpdirname, art.path))


        remote_client.set_terminated(remote_run_id)
        print('-'*10)

Processing experiment  0 Default
Experiment exists
Remote experiment id 0
Processing run: 8fe114316a26460aa2e463b8944cf230 -> 9847970ab991461394d6d1405f9ef2e6
Params: {}
created temporary directory /tmp/tmpsy03fsvu
----------
Processing run: e10f5cc7f8c44f079fd8214d4f14b840 -> 9a7aaf4d2cb6424c9151edf3e60400b2
Params: {}
created temporary directory /tmp/tmpose2v5ae
----------


In [260]:
run.info.start_time

1666599144446

In [188]:
run.data.params

{'name': 'resnet18.onnx', 'input_shape': '[3, 256, 256]'}

## remote client

In [169]:
remote_client = MlflowClient(tracking_uri='http://localhost:5000', registry_uri='http://localhost:5000')

In [170]:
remote_client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1666764720738, experiment_id='1', last_update_time=1666764720738, lifecycle_stage='active', name='remote2', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1666764686924, experiment_id='0', last_update_time=1666764686924, lifecycle_stage='active', name='Default', tags={}>]

In [172]:
remote_client.search_runs(experiment_ids=[0])

[<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.runName': 'auspicious-robin-946',
  'mlflow.source.git.commit': 'c95646ed5e23819cabe2538fc3a5e2cf5d7e179d',
  'mlflow.source.name': 'tt.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'jj'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/0/8101f3a0310f4fcb84b4b8fc104cac10/artifacts', end_time=1666764720799, experiment_id='0', lifecycle_stage='active', run_id='8101f3a0310f4fcb84b4b8fc104cac10', run_name='auspicious-robin-946', run_uuid='8101f3a0310f4fcb84b4b8fc104cac10', start_time=1666764720419, status='FINISHED', user_id='jj'>>]

In [173]:
remote_client.log_batch?

In [180]:
remote_client.search_experiments?

In [185]:
remote_client.search_experiments(filter_string="name = 'Default'")

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1666764686924, experiment_id='0', last_update_time=1666764686924, lifecycle_stage='active', name='Default', tags={}>]

In [187]:
remote_client.create_run(experiment_id='2')


<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.runName': 'adventurous-ant-385'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/2/9700734f7f34466b95e4569ace7c6f88/artifacts', end_time=None, experiment_id='2', lifecycle_stage='active', run_id='9700734f7f34466b95e4569ace7c6f88', run_name='adventurous-ant-385', run_uuid='9700734f7f34466b95e4569ace7c6f88', start_time=1667458567775, status='RUNNING', user_id='unknown'>>

In [203]:
remote_client.log_batch(run_id='9700734f7f34466b95e4569ace7c6f88',metrics=metric_history)

In [205]:
a_model

SimpleNet(
  (l1): Linear(in_features=784, out_features=10, bias=True)
)

In [209]:
artifacts = local_client.list_artifacts(run_id='e10f5cc7f8c44f079fd8214d4f14b840')

In [210]:
artifacts

[<FileInfo: file_size=432, is_dir=False, path='MLmodel'>,
 <FileInfo: file_size=199, is_dir=False, path='conda.yaml'>,
 <FileInfo: file_size=None, is_dir=True, path='data'>,
 <FileInfo: file_size=122, is_dir=False, path='python_env.yaml'>,
 <FileInfo: file_size=82, is_dir=False, path='requirements.txt'>]

In [248]:
mlflow.set_tracking_uri('http://localhost:5000')
with tempfile.TemporaryDirectory() as tmpdirname:
    print('created temporary directory', tmpdirname)
    for art in artifacts:
        local_client.download_artifacts(run_id='e10f5cc7f8c44f079fd8214d4f14b840', path=art.path, dst_path=tmpdirname)
        remote_client.log_artifact(run_id='9700734f7f34466b95e4569ace7c6f88', local_path=os.path.join(tmpdirname, art.path))

created temporary directory /tmp/tmp7eqwu79i
